In [2]:
import os, random, glob, pickle, collections, math
import numpy as np
import pandas as pd
import ujson as json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

In [3]:
# TRAIN_DIR = '../data/train/'
TEST_DIR = '../RFCN/JPEGImages/'
RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
CROP_MODEL = 'resnet50_DO5_FT38'
CHECKPOINT_DIR = './' + CROP_MODEL + '/checkpoint/'
if not os.path.exists(CHECKPOINT_DIR):
    os.mkdir(CHECKPOINT_DIR)
LOG_DIR = './' + CROP_MODEL + '/log/'
OUTPUT_DIR = './' + CROP_MODEL + '/output/'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BATCHSIZE = 128
LEARNINGRATE = 1e-4
BG_THRESH_HI = 0.3
BG_THRESH_LO = 0.1
# like RCNN expand the crop by 16 pixels
p=16

def load_img(path, bbox, target_size=None):
    img = Image.open(path)
    img = img.convert('RGB')
    cropped = img.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
    if target_size:
        cropped = cropped.resize((target_size[1], target_size[0]))
    return cropped

def preprocess_input(x):
    #resnet50 image preprocessing
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [4]:
# GTbbox_df = ['image_file','crop_index','crop_class','xmin',''ymin','xmax','ymax']

file_name = 'GTbbox_df.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    GTbbox_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)       
    GTbbox_df = pd.DataFrame(columns=['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax'])  

    crop_classes=FISH_CLASSES[:]
    crop_classes.remove('NoF')

    for c in crop_classes:
        print(c)
        j = json.load(open('../data/BBannotations/{}.json'.format(c), 'r'))
        for l in j: 
            filename = l["filename"]
            head, image_file = os.path.split(filename)
            basename, file_extension = os.path.splitext(image_file) 
            image = Image.open(TEST_DIR+'/'+image_file)
            width_image, height_image = image.size
            for i in range(len(l["annotations"])):
                a = l["annotations"][i]
                xmin = (a["x"])
                ymin = (a["y"])
                width = (a["width"])
                height = (a["height"])
                delta_width = p/(COLS-2*p)*width
                delta_height = p/(ROWS-2*p)*height
                xmin_expand = xmin-delta_width
                ymin_expand = ymin-delta_height
                xmax_expand = xmin+width+delta_width
                ymax_expand = ymin+height+delta_height
                assert max(xmin_expand,0)<min(xmax_expand,width_image)
                assert max(ymin_expand,0)<min(ymax_expand,height_image)
                GTbbox_df.loc[len(GTbbox_df)]=[image_file,i,a["class"],max(xmin_expand,0),max(ymin_expand,0),min(xmax_expand,width_image),min(ymax_expand,height_image)]
                if a["class"] != c: print(GTbbox_df.tail(1))

    GTbbox_df.to_pickle(OUTPUT_DIR+file_name)    

Loading from file GTbbox_df.pickle


In [5]:
# NoFbbox_df = ['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax',
#               'NoF_RFCN','ALB_RFCN','BET_RFCN','DOL_RFCN','LAG_RFCN','OTHER_RFCN','SHARK_RFCN','YFT_RFCN']
def IoU(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)
    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def bg_discrimination(box, gt_bboxes):
    # if BG_THRESH_LO<IoU<BG_THRESH_HI then box is bg return True
    IoUs = []
    for gt_box in gt_bboxes:
        IoUs.append(IoU(box, gt_box))
    if IoUs:
        if max(IoUs) > BG_THRESH_LO and max(IoUs) < BG_THRESH_HI:
            return True
        else:
            return False            

file_name = 'NoFbbox_df.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    NoFbbox_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)
    with open('../data/RFCN_detections/detections_RoIs_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_RoIs = pickle.load(f, encoding='latin1')
    train_detections_RoIs = detections_RoIs[1000:]
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    train_files = test_files[1000:]
    assert len(train_detections_RoIs) == len(train_files)

    NoFbbox_df = pd.DataFrame(columns=['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax',
                                       'NoF_RFCN','ALB_RFCN','BET_RFCN','DOL_RFCN','LAG_RFCN','OTHER_RFCN','SHARK_RFCN','YFT_RFCN']) 

    for im in range(len(train_files)):
        if im%1000 == 0: print(im)
        image_file = train_files[im][:-1]+'.jpg'
        image = Image.open(TEST_DIR+image_file)
        width_image, height_image = image.size
        gt_bboxes = GTbbox_df.loc[GTbbox_df['image_file'] == image_file, ['xmin','ymin','xmax','ymax']].values

        bboxes = []
        detects_im = train_detections_RoIs[im]
        for i in range(len(detects_im)):
            if bg_discrimination(detects_im[i,:4], gt_bboxes):
                bboxes.append(detects_im[i,:]) 

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            width = xmax-xmin
            height = ymax-ymin
            delta_width = p/(COLS-2*p)*width
            delta_height = p/(ROWS-2*p)*height
            xmin_expand = xmin-delta_width
            ymin_expand = ymin-delta_height
            xmax_expand = xmax+delta_width
            ymax_expand = ymax+delta_height
            assert max(xmin_expand,0)<min(xmax_expand,width_image)
            assert max(ymin_expand,0)<min(ymax_expand,height_image)
            NoFbbox_df.loc[len(NoFbbox_df)]=[image_file,j,'NoF',max(xmin_expand,0),max(ymin_expand,0),
                                             min(xmax_expand,width_image),min(ymax_expand,height_image)]+bbox[4:].tolist()
    
    NoFbbox_df.to_pickle(OUTPUT_DIR+file_name)

Loading from file NoFbbox_df.pickle


In [ ]:
# delete wrong samples
# NoFbbos_df.NoF_RFCN < 0.5

In [11]:
%%time
# train data prepare
# GTbbox_df = ['image_file','crop_index','crop_class','xmin',''ymin','xmax','ymax']
# NoFbbox_df = ['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax',
#               'NoF_RFCN','ALB_RFCN','BET_RFCN','DOL_RFCN','LAG_RFCN','OTHER_RFCN','SHARK_RFCN','YFT_RFCN']

train_fish_df, valid_fish_df = train_test_split(GTbbox_df, test_size=0.2, random_state=1986, stratify=GTbbox_df['crop_class'])
NoFbbox_df.drop(['NoF_RFCN','ALB_RFCN','BET_RFCN','DOL_RFCN','LAG_RFCN','OTHER_RFCN','SHARK_RFCN','YFT_RFCN'], axis=1, inplace=True)
nb_NoF_valid = math.ceil(valid_fish_df.shape[0]/10)
train_nof_df, valid_nof_df = train_test_split(NoFbbox_df, test_size=nb_NoF_valid, random_state=1986)

nb_fish = math.ceil(BATCHSIZE*0.25 / (len(FISH_CLASSES)-1))
nb_nof = BATCHSIZE - nb_fish*(len(FISH_CLASSES)-1)
samples_per_epoch=BATCHSIZE*math.ceil(train_fish_df.groupby('crop_class').size()['ALB']/nb_fish)
  
train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

def train_generator(datagen, fish_df, nof_df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(FISH_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_fish, replace=False),:]
        batch_fish_df = fish_df.groupby('crop_class', as_index=True).apply(fn)
        batch_nof_df = nof_df.sample(n=nb_nof)
        batch_df = pd.concat([batch_fish_df,batch_nof_df], axis=0)
        i = 0
        for index,row in batch_df.iterrows():
            image_file = row['image_file']
            fish = row['crop_class']
            bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
            cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,FISH_CLASSES.index(fish)] = 1
            i += 1
        yield (batch_x, batch_y)

# validation_data (valid_x,valid_y)
valid_df = pd.concat([valid_fish_df,valid_nof_df], axis=0)
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    image_file = row['image_file']
    fish = row['crop_class']
    bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
    cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
    x = np.asarray(cropped, dtype=K.floatx())
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,FISH_CLASSES.index(fish)] = 1
    i += 1


CPU times: user 16.6 s, sys: 220 ms, total: 16.8 s
Wall time: 16.7 s


In [6]:
#callbacks

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        

model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)


In [ ]:
#Resnet50
#top layer training

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
x = Dense(256, init='glorot_normal')(x)
x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), init='glorot_normal', activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=LEARNINGRATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator(datagen=train_datagen, fish_df=train_fish_df, nof_df=train_nof_df), 
                    samples_per_epoch=samples_per_epoch, nb_epoch=20, verbose=1,
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(valid_x,valid_y), nb_worker=3, pickle_safe=True)

Epoch 1/30
51456/51456 [==============================] - 709s - loss: 1.0746 - acc: 0.7141 - val_loss: 1.9061 - val_acc: 0.2679
Epoch 2/30
51456/51456 [==============================] - 657s - loss: 0.6876 - acc: 0.7870 - val_loss: 1.8943 - val_acc: 0.2876
Epoch 3/30
51456/51456 [==============================] - 673s - loss: 0.5996 - acc: 0.8092 - val_loss: 1.7672 - val_acc: 0.3292
Epoch 4/30
51456/51456 [==============================] - 674s - loss: 0.5509 - acc: 0.8218 - val_loss: 1.6744 - val_acc: 0.3499
Epoch 5/30
51456/51456 [==============================] - 672s - loss: 0.5280 - acc: 0.8295 - val_loss: 1.5273 - val_acc: 0.3988
Epoch 6/30
51456/51456 [==============================] - 672s - loss: 0.5008 - acc: 0.8341 - val_loss: 1.4981 - val_acc: 0.4071
Epoch 7/30
51456/51456 [==============================] - 672s - loss: 0.4847 - acc: 0.8388 - val_loss: 1.4942 - val_acc: 0.4112
Epoch 8/30
51456/51456 [==============================] - 673s - loss: 0.4737 - acc: 0.8431 - val

In [12]:
# Resnet50
# fine tuning
# 164 conv5c+top
# 142 conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
start_layer = 38

files = glob.glob(CHECKPOINT_DIR+'*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model('./checkpoints/checkpoint3/weights.004-0.0565.hdf5')

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator(datagen=train_datagen, fish_df=train_fish_df, nof_df=train_nof_df),
                    samples_per_epoch=samples_per_epoch, nb_epoch=300, verbose=1,
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(valid_x,valid_y), nb_worker=3, pickle_safe=True)

Loading model from checkpoints file ./checkpoints/checkpoint_resnet50_DO5_256_DO5/weights.003-1.2276.hdf5
Epoch 1/300
51456/51456 [==============================] - 1268s - loss: 0.2979 - acc: 0.8955 - val_loss: 1.0087 - val_acc: 0.6127
Epoch 2/300
51456/51456 [==============================] - 1233s - loss: 0.2030 - acc: 0.9297 - val_loss: 0.8053 - val_acc: 0.7009
Epoch 3/300
51456/51456 [==============================] - 1234s - loss: 0.1533 - acc: 0.9466 - val_loss: 0.7218 - val_acc: 0.7425
Epoch 4/300
51456/51456 [==============================] - 1234s - loss: 0.1189 - acc: 0.9596 - val_loss: 0.6657 - val_acc: 0.7757
Epoch 5/300
51456/51456 [==============================] - 1233s - loss: 0.0966 - acc: 0.9674 - val_loss: 0.5854 - val_acc: 0.8079
Epoch 6/300
51456/51456 [==============================] - 1233s - loss: 0.0787 - acc: 0.9731 - val_loss: 0.5331 - val_acc: 0.8359
Epoch 7/300
51456/51456 [==============================] - 1233s - loss: 0.0642 - acc: 0.9787 - val_loss: 0.

In [7]:
#resume training

files = glob.glob(CHECKPOINT_DIR+'*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
model = load_model(files[index])
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model('./checkpoints/checkpoint3/weights.004-0.0565.hdf5')

model.fit_generator(train_generator(datagen=train_datagen, fish_df=train_fish_df, nof_df=train_nof_df),
                    samples_per_epoch=samples_per_epoch, nb_epoch=300, verbose=1,
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(valid_x,valid_y), nb_worker=3, pickle_safe=True)

Loading model from checkpoints file ./checkpoints/checkpoint_resnet50_DO5_256_DO5/weights.015-1.2579.hdf5
Epoch 1/300
51456/51456 [==============================] - 706s - loss: 0.4161 - acc: 0.8593 - val_loss: 1.2661 - val_acc: 0.4912
Epoch 2/300
51456/51456 [==============================] - 661s - loss: 0.4058 - acc: 0.8625 - val_loss: 1.2640 - val_acc: 0.4881
Epoch 3/300
51456/51456 [==============================] - 672s - loss: 0.4065 - acc: 0.8619 - val_loss: 1.2951 - val_acc: 0.4808
Epoch 4/300
51456/51456 [==============================] - 672s - loss: 0.4013 - acc: 0.8649 - val_loss: 1.2276 - val_acc: 0.5088
Epoch 5/300
51456/51456 [==============================] - 670s - loss: 0.3981 - acc: 0.8659 - val_loss: 1.2436 - val_acc: 0.4984
Epoch 6/300
51456/51456 [==============================] - 673s - loss: 0.3886 - acc: 0.8670 - val_loss: 1.2716 - val_acc: 0.4912
Epoch 7/300
51456/51456 [==============================] - 672s - loss: 0.3943 - acc: 0.8664 - val_loss: 1.2487 - 

In [6]:
#test prepare

files = glob.glob(CHECKPOINT_DIR+'*')
val_losses = [float(f.split('-')[-1][:-5]) for f in files]
index = val_losses.index(min(val_losses))
print('Loading model from checkpoints file ' + files[index])
test_model = load_model(files[index])
# print('Loading model from weights.004-0.0565.hdf5')
# test_model = load_model('./checkpoints/checkpoint2/weights.004-0.0565.hdf5')
test_model_name = files[index].split('/')[-1]

def test_generator(df, datagen = None, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            image_file = row['image_file']
            bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
            cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            if datagen is not None: x = datagen.random_transform(x)            
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%50 == 0: print(batch_index)
        yield(batch_x)
        
test_aug_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

Loading model from checkpoints file ./resnet50_DO5_FT38/checkpoint/weights.025-0.3670.hdf5


In [8]:
#GTbbox_CROPpred_df = ['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax',
#                      'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'logloss']

file_name = 'GTbbox_CROPpred_df_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_CROPpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name) 
    nb_augmentation = 1
    if nb_augmentation ==1:
        test_preds = test_model.predict_generator(test_generator(df=GTbbox_df), val_samples=GTbbox_df.shape[0], nb_worker=1, pickle_safe=False)
    else:
        test_preds = np.zeros((GTbbox_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
        for idx in range(nb_augmentation):
            print('{}th augmentation for testing ...'.format(idx+1))
            test_preds += test_model.predict_generator(test_generator(df=GTbbox_df,datagen=test_aug_datagen), 
                                                       val_samples=GTbbox_df.shape[0], nb_worker=1, pickle_safe=False)
        test_preds /= nb_augmentation

    columns = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
    CROPpred_df = pd.DataFrame(test_preds, columns=columns)
    GTbbox_CROPpred_df = pd.concat([GTbbox_df,CROPpred_df], axis=1)
    GTbbox_CROPpred_df['logloss'] = GTbbox_CROPpred_df.apply(lambda row: -math.log(row[row['crop_class']]), axis=1)
    GTbbox_CROPpred_df.to_pickle(OUTPUT_DIR+file_name) 

#logloss of every fish class
print(GTbbox_CROPpred_df.groupby(['crop_class'])['logloss'].mean())
print(GTbbox_CROPpred_df['logloss'].mean())
# logloss of NoF
NoFbbox_df_sample = NoFbbox_df.sample(n=10000)
nof_preds = test_model.predict_generator(test_generator(df=NoFbbox_df_sample), val_samples=NoFbbox_df_sample.shape[0], nb_worker=1, pickle_safe=False)
print('NoF      ', np.mean(-np.log(nof_preds[:,0])))

Loading from file GTbbox_CROPpred_df_weights.025-0.3670.hdf5_.pickle
crop_class
ALB      0.140953
BET      0.051831
DOL      0.189900
LAG      0.000063
OTHER    0.054545
SHARK    0.077625
YFT      0.103938
Name: logloss, dtype: float64
0.11665313641105421
50
0
NoF       0.00436714


In [ ]:
# resnet50_DO5_256_DO5_FT38
Loading from file GTbbox_CROPpred_df_weights.012-0.3948.hdf5_.pickle
crop_class
ALB      0.251183
BET      0.100649
DOL      0.181231
LAG      0.000128
OTHER    0.059655
SHARK    0.073495
YFT      0.112141
Name: logloss, dtype: float64
0.18490654219054167
50
0
NoF       0.0122033

In [9]:
# RFCNbbox_RFCNpred_df = ['image_class','image_file','crop_index','xmin','ymin','xmax','ymax',
#                          'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                          'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']

file_name = 'RFCNbbox_RFCNpred_df.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)        
    RFCNbbox_RFCNpred_df = pd.DataFrame(columns=['image_class','image_file','crop_index','xmin','ymin','xmax','ymax',
                                                  'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
                                                  'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']) 
#     RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_file_labels = f.readlines()
    assert len(detections_full_AGNOSTICnms) == len(test_files)

    for im in range(len(detections_full_AGNOSTICnms)):
        if im%1000 == 0: print(im)
        basename = test_files[im][:9]
        if im<1000:
            image_class = '--'
        else:
            for i in range(len(train_file_labels)):
                if train_file_labels[i][:9] == basename:
                    image_class = train_file_labels[i][10:-1]
                    break
        image = Image.open(TEST_DIR+'/'+basename+'.jpg')
        width_image, height_image = image.size
        
        bboxes = []
        detects_im = detections_full_AGNOSTICnms[im]
        for i in range(len(detects_im)):
            if np.max(detects_im[i,5:]) >= CONF_THRESH:
                bboxes.append(detects_im[i,:]) 
        if len(bboxes) == 0:
            ind = np.argmax(np.max(detects_im[:,5:], axis=1))
            bboxes.append(detects_im[ind,:])
        bboxes = np.asarray(bboxes)

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            width = xmax-xmin
            height = ymax-ymin
            delta_width = p/(COLS-2*p)*width
            delta_height = p/(ROWS-2*p)*height
            xmin_expand = xmin-delta_width
            ymin_expand = ymin-delta_height
            xmax_expand = xmax+delta_width
            ymax_expand = ymax+delta_height
            assert max(xmin_expand,0)<min(xmax_expand,width_image)
            assert max(ymin_expand,0)<min(ymax_expand,height_image)
            RFCNbbox_RFCNpred_df.loc[len(RFCNbbox_RFCNpred_df)]=[image_class,basename+'.jpg',j,max(xmin_expand,0),max(ymin_expand,0),
                                                                   min(xmax_expand,width_image),min(ymax_expand,height_image),
                                                                   bbox[4],bbox[5],bbox[6],bbox[7],bbox[8],bbox[9],bbox[10],bbox[11]]   
    
    RFCNbbox_RFCNpred_df.to_pickle(OUTPUT_DIR+file_name)      

Loading from file RFCNbbox_RFCNpred_df.pickle


In [10]:
# RFCNbbox_RFCNpred_CROPpred_df = ['image_class', 'image_file','crop_index','x_min','y_min','x_max','ymax',
#                                    'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                                    'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN'
#                                    'NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP',
#                                    'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP']

file_name = 'RFCNbbox_RFCNpred_CROPpred_df_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_CROPpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)  
    nb_augmentation = 5
    if nb_augmentation ==1:
        test_preds = test_model.predict_generator(test_generator(df=RFCNbbox_RFCNpred_df), 
                                                  val_samples=RFCNbbox_RFCNpred_df.shape[0], nb_worker=1, pickle_safe=False)
    else:
        test_preds = np.zeros((RFCNbbox_RFCNpred_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
        for idx in range(nb_augmentation):
            print('{}th augmentation for testing ...'.format(idx+1))
            test_preds += test_model.predict_generator(test_generator(df=RFCNbbox_RFCNpred_df,datagen=test_aug_datagen), 
                                                       val_samples=RFCNbbox_RFCNpred_df.shape[0], nb_worker=1, pickle_safe=False)
        test_preds /= nb_augmentation

    columns = ['NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP', 'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP']
    CROPpred_df = pd.DataFrame(test_preds, columns=columns)
    RFCNbbox_RFCNpred_CROPpred_df = pd.concat([RFCNbbox_RFCNpred_df,CROPpred_df], axis=1)
    RFCNbbox_RFCNpred_CROPpred_df.to_pickle(OUTPUT_DIR+file_name) 

Generating file RFCNbbox_RFCNpred_CROPpred_df_weights.025-0.3670.hdf5_.pickle
1th augmentation for testing ...
0
2th augmentation for testing ...
0
3th augmentation for testing ...
0
4th augmentation for testing ...
0
5th augmentation for testing ...
0


In [12]:
# clsMaxAve
# test_pred_df = ['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    test_files = f.readlines()
    
test_pred_df = pd.DataFrame(columns=['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT'])  
for j in range(len(test_files)):
    image_file = test_files[j][:-1]+'.jpg'
    test_pred_im_df = RFCNbbox_RFCNpred_CROPpred_df.loc[RFCNbbox_RFCNpred_CROPpred_df['image_file'] == image_file, \
                                                        ['image_class','NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP', 'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP']] 
    image_class = test_pred_im_df.iloc[0]['image_class']
    test_pred_im_df.drop('image_class', axis=1, inplace=True)
    max_score = test_pred_im_df.max(axis=1)
    max_cls = test_pred_im_df.idxmax(axis=1)
    test_pred_im_df['max_score'] = max_score
    test_pred_im_df['max_cls'] = max_cls
    test_pred_im_df['Count'] = test_pred_im_df.groupby(['max_cls'])['max_cls'].transform('count')
    idx = test_pred_im_df.groupby(['max_cls'])['max_score'].transform(max) == test_pred_im_df['max_score']
    test_pred_im_clsMax_df = test_pred_im_df.loc[idx,['NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP', 'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP', 'Count']]
    test_pred_im_clsMax_array = test_pred_im_clsMax_df.values
    pred = np.average(test_pred_im_clsMax_array[:,:-1], axis=0, weights=test_pred_im_clsMax_array[:,-1], returned=False).tolist()
    if image_class!='--':
        ind = FISH_CLASSES.index(image_class)
        logloss = -math.log(pred[ind]) 
    else:
        logloss = np.nan
    test_pred_im_clsMaxAve = [logloss,image_class,image_file]
    test_pred_im_clsMaxAve.extend(pred)
    test_pred_df.loc[len(test_pred_df)]=test_pred_im_clsMaxAve

In [14]:
#temperature
T = 2.5
test_pred_array = test_pred_df[FISH_CLASSES].values
test_pred_T_array = np.exp(np.log(test_pred_array)/T)
test_pred_T_array = test_pred_T_array/np.sum(test_pred_T_array, axis=1, keepdims=True)
test_pred_T_df = pd.DataFrame(test_pred_T_array, columns=FISH_CLASSES)
test_pred_T_df = pd.concat([test_pred_df[['image_class','image_file']],test_pred_T_df], axis=1)

#add logloss
test_pred_T_df['logloss'] = test_pred_T_df.apply(lambda row: -math.log(row[row['image_class']]) if row['image_class']!='--' else np.nan, axis=1)

#calculate train logloss
print(test_pred_T_df.groupby(['image_class'])['logloss'].mean())
train_logloss = test_pred_T_df['logloss'].mean()
print('logloss of train is', train_logloss )

image_class
--            NaN
ALB      0.201983
BET      0.114692
DOL      0.137755
LAG      0.025413
NoF      0.171720
OTHER    0.067205
SHARK    0.118979
YFT      0.110365
Name: logloss, dtype: float64
logloss of train is 0.15617129881332498


In [11]:
image_class
--            NaN
ALB      0.379154
BET      0.150810
DOL      0.111402
LAG      0.020587
NoF      0.191428
OTHER    0.088848
SHARK    0.087299
YFT      0.170171
Name: logloss, dtype: float64
logloss of train is 0.2521023066961748

SyntaxError: invalid syntax (<ipython-input-11-a37b2b1f3545>, line 3)

In [15]:
#test submission
submission = test_pred_T_df.loc[:999,['image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']]
submission.rename(columns={'image_file':'image'}, inplace=True)
sub_file = 'RFCN_AGONOSTICnms_'+RFCN_MODEL+'_'+CROP_MODEL+'_clsMaxAve_conf{:.2f}_T{}_'.format(CONF_THRESH, T)+'{:.4f}'.format(train_logloss)+'.csv'
submission.to_csv(sub_file, index=False)
print('Done!'+sub_file)

Done!RFCN_AGONOSTICnms_resnet101_rfcn_ohem_iter_30000_resnet50_DO5_FT38_clsMaxAve_conf0.80_T2.5_0.1562.csv
